In [1]:
%load_ext autoreload
%autoreload 2
%autoreload 2

In [5]:
import numpy as np
import pandas as pd
import plotly.express as px

from datetime import datetime
from dateutil.tz import gettz
import os
import time

In [6]:
def line_scatter(df, x, y, **kwargs):
    fig = px.line(df, x=x, y=y, **kwargs, template="plotly_white")
    for data in fig.data:
        data.update(mode="markers+lines")
    return fig